# Structured Outputs

LLMs regurgitate out text and that is great for so many applications. But in order to build strong, robust systems and applications, we need to make sense of the chaos sometimes by receiving a pre-determined structured output everytime an LLM is called.

## As always, libraries first!

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# check if API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Missing OpenAI API key")
if not GEMINI_API_KEY:
    raise ValueError("Missing Gemini API key")
if not ANTHROPIC_API_KEY:
    raise ValueError("Missing Anthropic API key")

## Introducing the Pydantic library

In [2]:
from pydantic import BaseModel

class CustomerTicket(BaseModel):
    ticket: str
    priority: str

class TicketResponse(BaseModel):
    response: str
    resolution_time: str

class TicketEvaluation(BaseModel):
    passed: bool
    feedback: str

## Calling OpenAI to generate support tickets

In [3]:
user_message = "I want you to generate a customer support ticket for a 3rd party tech re-seller. "
user_message += "The ticket should be a single sentence describing a common issue a customer might face with their product or service. "
user_message += "Please ensure the ticket is varied and covers different types of problems."

messages = [{"role": "user", "content": user_message}]

In [4]:
client = OpenAI()

openai_response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=CustomerTicket
)

tickets = openai_response.choices[0].message.parsed

# display(Markdown(tickets))

In [5]:
tickets.ticket

"The customer's printer is not responding when attempting to print documents."

In [6]:
tickets.priority

'High'

## An evaluator-optimizer workflow

In [7]:
message = "You are to propose a resolution for the following customer support ticket. \n\n"
message += f"Ticket: {tickets.ticket}\n"
message += f"Priority: {tickets.priority}\n\n"

messages = [{"role": "user", "content": message}]

In [8]:
response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=TicketResponse
)

response = response.choices[0].message.parsed

In [11]:
display(Markdown(response.response))

Thank you for bringing this issue to our attention. To resolve the printer not responding, please follow these steps:

1. Ensure the printer is powered on and connected properly to your computer or network.
2. Check for any error messages or indicator lights on the printer.
3. Restart the printer and your computer.
4. Verify that the printer drivers are up to date.
5. If using a network printer, ensure it is connected to the network and accessible.

If the issue persists after these steps, please provide the printer model and any error messages displayed, so we can assist further. We understand this is a high-priority issue and will do our best to resolve it promptly.

In [12]:
display(Markdown(response.resolution_time))

Within 4 hours of receiving the ticket.

In [17]:
response.resolution_time

'Within 4 hours of receiving the ticket.'

## Lets evaluate our response

In [19]:
messages

[{'role': 'user',
  'content': "You are to evaluate the proposed resolution for the following customer support ticket. You will determine if the proposed resolution is appropriate for the ticket and priority level. tickets\n\nTicket: The customer's printer is not responding when attempting to print documents.\nPriority: High\n\nProposed Resolution: Thank you for bringing this issue to our attention. To resolve the printer not responding, please follow these steps:\n\n1. Ensure the printer is powered on and connected properly to your computer or network.\n2. Check for any error messages or indicator lights on the printer.\n3. Restart the printer and your computer.\n4. Verify that the printer drivers are up to date.\n5. If using a network printer, ensure it is connected to the network and accessible.\n\nIf the issue persists after these steps, please provide the printer model and any error messages displayed, so we can assist further. We understand this is a high-priority issue and will 

In [ ]:
message = "You are to evaluate the proposed resolution for the following customer support ticket. "
message += "You will determine if the proposed resolution is appropriate for the ticket and priority level. "
message += "tickets\n\n"
message += f"Ticket: {tickets.ticket}\n"
message += f"Priority: {tickets.priority}\n\n"
message += f"Proposed Resolution: {response.response}\n"
message += f"Proposed Resolution Time: {response.resolution_time}\n\n"

messages = [{"role": "user", "content": message}]

In [28]:
evaluate = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=TicketEvaluation
)

evaluate = evaluate.choices[0].message.parsed

In [29]:
print("Pass or not: ", evaluate.passed)
print("Feedback: ", evaluate.feedback)

Pass or not:  False
Feedback:  The proposed resolution provides general troubleshooting steps suitable for addressing a non-responsive printer; however, the suggested resolution time of 12 days is too lengthy for a high-priority issue. High-priority tickets typically require immediate or prompt attention to minimize disruption. It would be more appropriate to expedite the resolution process, perhaps aiming for initial troubleshooting within 1-2 days, and providing a clear, urgent support plan to the customer.
